<a href="https://colab.research.google.com/github/MarkLimmage/AustralianSharePrices/blob/main/stock_history.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm.notebook import tqdm

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import dateutil.parser as dp
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
motley_3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MA5851/assessments/A2/motley_3.csv')

In [ ]:
allords = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MA5851/assessments/A2/allords.csv', encoding = "ISO-8859-1"))
allords

,code,company
0,360,LIFE360 Inc
1,3DP,Pointerra Ltd
2,A200,Betashares Australia 200 ETF
3,A2M,The a2 Milk Company Ltd
4,A4N,Alpha Hpa Ltd
...,...,...
493,XRO,Xero Ltd
494,YAL,Yancoal Australia Ltd
495,Z1P,ZIP Co Ltd
496,ZEL,Z Energy Ltd


In [ ]:
url = 'https://au.finance.yahoo.com/quote/{}/history?period1=1577836800&period2=1661040000&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true'

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36' }

soup = dict()

articles = pd.DataFrame( columns=['company', 'article_url', 'pub_time'])
errors = pd.DataFrame( columns=['company', 'script'] )
a_prices = pd.DataFrame()

for index, row in tqdm(allords.iterrows(), total=allords.shape[0]):
  print(row['code'])
  company = str(row['code']) + '.AX'
  page = url.format(company)
  response = requests.get(page, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  pattern = re.compile(r'\s--\sData\s--\s')
  script_data=soup.find('script', text=pattern).contents[0]
  data = json.loads(script_data[112:-12])
  try:
    s_prices = data['context']['dispatcher']['stores']['HistoricalPriceStore']['prices']
    s_prices = pd.DataFrame.from_dict(s_prices, orient='columns')
    s_prices.set_index(s_prices['date'], drop=True, append=False, inplace=True, verify_integrity=False)
    s_prices.drop(['date','open','high','low','volume','adjclose'], axis=1, inplace=True)
    s_prices = s_prices.rename({'close':str(row['code'])}, axis=1)
    a_prices = pd.concat([a_prices, s_prices], axis=1)
  except:
    pass
      


  0%|          | 0/498 [00:00<?, ?it/s]

360
3DP
A200
A2M
A4N
AAA
AAC
ABB
ABC
ABP
ABR
ABY
AD8
ADH
ADI
ADN
ADO
ADT
AEF
AFG
AFI
AFP
AGG
AGL
AHY
AIA
AIZ
AKP
ALC
ALD
ALG
ALK
ALL
ALQ
ALU
ALX
AMA
AMC
AMI
AMP
ANN
ANZ
AOF
APA
APE
API
APL
APT
APX
AQR
AQZ
ARB
ARF
ARG
ART
ASB
ASG
ASIA
ASM
AST
ASX
AUB
AUI
AVN
AVZ
AWC
AX1
AZJ
BAP
BBN
BCK
BEN
BET
BFG
BGA
BGL
BGP
BHP
BILL
BIN
BKI
BKL
BKW
BLD
BLX
BOQ
BPT
BRG
BRN
BSL
BTH
BVS
BWP
BWX
BXB
CAJ
CAR
CAT
CBA
CCL
CCP
CCX
CDA
CEN
CGC
CGF
CHC
CHN
CIA
CIM
CIN
CIP
CKF
CLW
CMM
CMW
CNI
CNU
COE
COF
COH
COL
CPU
CQE
CQR
CRED
CRN
CSL
CSR
CTD
CTT
CUV
CVN
CWN
CWP
CWY
CXL
DBI
DDH
DDR
DEG
DHG
DJRE
DJW
DMP
DOW
DRR
DTL
DUB
DUI
DXS
EBO
ECX
EHE
EHL
ELD
ELO
EML
EMR
EOF
EOS
EQT
ERA
ERD
ETHI
EVN
EVT
FAIR
FBU
FCL
FDV
FGG
FGX
FLN
FLT
FMG
FPH
FSF
GCI
GDI
GDX
GEM
GMA
GMG
GNC
GNE
GOLD
GOR
GOZ
GPT
GRR
GUD
GWA
GXY
HACK
HBRD
HDN
HGH
HLS
HM1
HMC
HPI
HSN
HT1
HTA
HUB
HUM
HVN
HYGG
IAA
IAF
IAG
IAP
IDX
IEL
IEM
IEU
IFL
IFM
IFRA
IFT
IGO
IHVV
IJP
ILC
ILU
IMD
IMU
INA
ING
INR
IOO
IOZ
IPH
IPL
IRE
IRI
ISX
IVC
IVE
IVV
IXJ
JBH
JHG
JHX
JIN
JLG

In [ ]:
s_prices = data['context']['dispatcher']['stores']['HistoricalPriceStore']['prices']
s_prices = pd.DataFrame.from_dict(s_prices, orient='columns')
s_prices.set_index(s_prices['date'], drop=True, append=False, inplace=True, verify_integrity=False)
s_prices.drop(['date','open','high','low','volume','adjclose'], axis=1, inplace=True)
s_prices = s_prices.rename({'close':str(row['code'])}, axis=1)
a_prices[str(row['code'])] = s_prices[str(row['code'])]


,3DP
date,
1660867200,0.215
1660780800,0.220
1660694400,0.235
1660608000,0.225
1660521600,0.225
...,...
1578438000,0.055
1578351600,0.057
1578265200,0.054


In [ ]:
a_prices

a_prices.to_csv('/content/drive/MyDrive/Colab Notebooks/MA5851/assessments/A2/a_prices.csv')

In [ ]:
"""Extract nested values from a JSON tree."""


def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

json_extract( data, 'HistoricalPriceStore' )


[]

In [ ]:
url = 'https://au.finance.yahoo.com/quote/{}/history?period1=1577836800&period2=1661040000&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true'

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36' }

soup = dict()

articles = pd.DataFrame( columns=['company', 'article_url', 'pub_time'])
errors = pd.DataFrame( columns=['company', 'script'] )
a_prices = pd.DataFrame()

company = str('360') + '.AX'
page = url.format(company)
response = requests.get(page, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(r'\s--\sData\s--\s')
script_data=soup.find('script', text=pattern).contents[0]
data = json.loads(script_data[112:-12])

s_prices = data['context']['dispatcher']['stores']['HistoricalPriceStore']['prices']
s_prices = pd.DataFrame.from_dict(s_prices, orient='columns')
s_prices.set_index(s_prices['date'], drop=True, append=False, inplace=True, verify_integrity=False)
data

1660867200

{'context': {'dispatcher': {'stores': {'PageStore': {'currentPageName': 'quote',
     'currentEvent': {'eventName': 'NEW_PAGE_SUCCESS'},
     'currentRenderTargetId': 'default',
     'pagesConfigRaw': {'base': {'quote': {'layout': {'bundleName': 'yahoodotcom-layout.TwoColumnLayout',
         'name': 'TwoColumnLayout',
         'config': {'enableHeaderCollapse': True,
          'additionalBodyWrapperClasses': 'Bgc($layoutBgColor)!',
          'contentWrapperClasses': 'Bgc($lv2BgColor)!',
          'Header': {'isFixed': True,
           'uhContainerClasses': 'Bgi($uhGrayGradient) D(n)--print',
           'navContainerClasses': 'Bgi($navrailGrayGradient) Bxsh($navrailShadow) Pos(r) hasScrolled_Bxsh(headerShadow) Panel-open_Bxsh(headerShadow)',
           'navTransitionClasses': 'HideNavrail_Translate3d(0,-46px,0) Panel-open_Translate3d(0,-46px,0)',
           'secondaryNavContainerClasses': 'hasScrolled_Bdbw(0px) Bgc($lv3BgColor) Bxsh($navrailShadow) D(n)--print',
           'height': 135

In [ ]:
s_prices.drop(['date','open','high','low','volume','adjclose'], axis=1, inplace=True)
s_prices = s_prices.rename({'close':str(row['code'])}, axis=1)
a_prices[str(row['code'])] = s_prices[str(row['code'])]